In [10]:
#Importing needed libraries
import random 
from time import sleep
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np

In [11]:
#open the webpage
url = 'https://itdashboard.gov/'
driver = webdriver.Chrome('./chromedriver')
driver.get(url)

In [12]:
#click the button
button = driver.find_elements_by_css_selector("[aria-controls=home-dive-in]")
print(button[0])
button[0].click()
time.sleep(5)

<selenium.webdriver.remote.webelement.WebElement (session="dcee801129fbb824bef88fa6ea613d91", element="d59ffbbc-4c41-4bfa-9921-8e755be4055a")>


In [13]:
#access the main information from the departments
departments = driver.find_elements_by_id('agency-tiles-container')
a = departments[0]
a = a.find_element_by_id('agency-tiles-widget')
a = a.find_elements_by_class_name('wrapper')
b = a[0]
dep_list_p = b.text
dep_list = dep_list_p.split('\n')
dep_dict = {}
print(dep_list)


['Department of Agriculture', 'Total FY2021 Spending:', '$2.7B', 'view', 'Department of Commerce', 'Total FY2021 Spending:', '$2.8B', 'view', 'Department of Defense', 'Total FY2021 Spending:', '$36B', 'view', 'Department of Health and Human Services', 'Total FY2021 Spending:', '$7.0B', 'view', 'Department of the Interior', 'Total FY2021 Spending:', '$1.5B', 'view', 'Department of Justice', 'Total FY2021 Spending:', '$3.1B', 'view', 'Department of Labor', 'Total FY2021 Spending:', '$819M', 'view', 'Department of State', 'Total FY2021 Spending:', '$2.6B', 'view', 'Department of the Treasury', 'Total FY2021 Spending:', '$5.4B', 'view', 'Social Security Administration', 'Total FY2021 Spending:', '$2.0B', 'view', 'Department of Education', 'Total FY2021 Spending:', '$1.0B', 'view', 'Department of Energy', 'Total FY2021 Spending:', '$3.1B', 'view', 'Environmental Protection Agency', 'Total FY2021 Spending:', '$385M', 'view', 'Department of Transportation', 'Total FY2021 Spending:', '$3.5B', 

In [23]:
#Generating dataframe with the information collected
for i in range(len(dep_list)-2):
    if i%4 == 0:                            #Module=0 as the 0 residual of the divission, multiples of 4
        dep_dict[dep_list[i]] = dep_list[i+2]


print(len(dep_dict))
dep_dict

data_final = pd.DataFrame(data = dep_dict, index=[0])
data_final = (data_final.T)
print(data_final)

26
                                                    0
Department of Agriculture                       $2.7B
Department of Commerce                          $2.8B
Department of Defense                            $36B
Department of Health and Human Services         $7.0B
Department of the Interior                      $1.5B
Department of Justice                           $3.1B
Department of Labor                             $819M
Department of State                             $2.6B
Department of the Treasury                      $5.4B
Social Security Administration                  $2.0B
Department of Education                         $1.0B
Department of Energy                            $3.1B
Environmental Protection Agency                 $385M
Department of Transportation                    $3.5B
General Services Administration                 $765M
Department of Homeland Security                 $7.4B
Department of Housing and Urban Development     $447M
National Aeronautics and 

In [15]:
#xpath format 
#--------------------------------------------------------------------#
#//*[@id="agency-tiles-widget"]/div/div[1]/div[1]/div/div/div/div[2]/a
#//*[@id="agency-tiles-widget"]/div/div[1]/div[2]/div/div/div/div[2]/a
#//*[@id="agency-tiles-widget"]/div/div[9]/div[2]/div/div/div/div[2]/a
#--------------------------------------------------------------------#
#Choosing a random dept. and redirecting 
index = random.randint(0,len(dep_dict)-1)
department_chosen = list (dep_dict.keys())[index]
print(department_chosen, index)

button_2 = driver.find_elements_by_xpath('//*[@id="agency-tiles-widget"]/div/div['+str(index//3 + 1 )+']/div['+str(index%3 +1) +']/div/div/div/div[2]/a')
page_2 = button_2[0].click()
time.sleep(9)

Small Business Administration 19


In [16]:
#Scraping individual investments and gathering information in a dataframe

titles = driver.find_elements_by_xpath('//th')
df_headers=[]

for t in titles:
    if t.text !='':
        df_headers.append(t.text)

table_content = driver.find_elements_by_xpath('//*[@id="investments-table-object"]/tbody/tr/td')
cols_no = len(df_headers)
rows = len(table_content)/cols_no
shape = (int(rows), int(cols_no))
df = pd.DataFrame(np.zeros(shape),columns=[df_headers])

for i in range(len(table_content)):
    df.iloc[int(i%rows), int(i%cols_no)] = table_content[i].text

df.head(5)

In [18]:
#Accesing links and downloading attached pdf's if possible
url_3 = 'https://itdashboard.gov/drupal/summary/'
for uii in list(df.iloc[:,0]):
    try:
        time.sleep(5)
        str_uii = str(uii)
        head = str_uii.split('-')[0]
        url_temp = url_3 + head + '/' + str_uii
        print(url_temp)
        driver.get(url_temp)
        pdf_download = driver.find_element_by_id('business-case-pdf')
        pdf_download.click()
        time.sleep(5)
    except:
        print('No file available for download')


https://itdashboard.gov/drupal/summary/028/028-000000048
No file available for download
https://itdashboard.gov/drupal/summary/028/028-000000045
https://itdashboard.gov/drupal/summary/028/028-000005001
No file available for download
https://itdashboard.gov/drupal/summary/028/028-000002001
No file available for download
https://itdashboard.gov/drupal/summary/028/028-000001005
No file available for download
https://itdashboard.gov/drupal/summary/028/028-000001004
https://itdashboard.gov/drupal/summary/028/028-000000070
https://itdashboard.gov/drupal/summary/028/028-000000065
https://itdashboard.gov/drupal/summary/028/028-000000056
https://itdashboard.gov/drupal/summary/028/028-000008007


In [24]:
#Generating final report in xlsx format. 
with pd.ExcelWriter('output.xlsx') as writer:  
    data_final.to_excel(writer, sheet_name='Agencies', header = False)
    df.to_excel(writer, sheet_name='Individual Investments')